In [1]:
# Load model directly
#8496
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
# tokenizer = AutoTokenizer.from_pretrained("/work/frink/models/alpaca-13b")
# model = AutoModelForCausalLM.from_pretrained("/work/frink/models/alpaca-13b", torch_dtype=torch.bfloat16, 
#     device_map = "auto")
# model = model.to('cuda')
from tqdm import tqdm
import string

In [3]:

read_path = '/home/ramprasad.sa/factual_evaluation_source_based/datasets/scored/xformer_llm_annotated_scored_label.csv'
# write_path = '/home/ramprasad.sa/factual_evaluation_source_based/datasets/scored/xformer_llm_annotated_scored_label.csv'
df_scored = pd.read_csv(read_path)
df_scored.head()[:1]


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,docid,model,nonfactual_spans,evidence,summary,factual_error,error_type,...,ChatGPT-direct_assesment_response,ChatGPT-direct_assesment_label,ChatGPT-direct_assesment_response_afacts,ChatGPT-direct_assesment_label_afacts,ChatGPT-direct_assessment_score_response,ChatGPT-direct_assessment_score_label,ChatGPT-direct_assessment_score_response_afacts,ChatGPT-direct_assessment_score_label_afacts,Alpaca-direct_assesment_response,Alpaca-direct_assesment_label
0,0,40,40,13681241,MV-BART,[],[],Callum is still busy and sorry.,0,[],...,yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1


In [9]:
#### nonfactual examples ###
df_scored_nonfactual = df_scored[df_scored['factual_error'] == 1]
df_sample_nonfactual = df_scored_nonfactual.sample(2)
drop_idx = list(df_sample_nonfactual.index)
df_scored.drop(index=drop_idx)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,docid,model,nonfactual_spans,evidence,summary,factual_error,error_type,...,ChatGPT-direct_assesment_response,ChatGPT-direct_assesment_label,ChatGPT-direct_assesment_response_afacts,ChatGPT-direct_assesment_label_afacts,ChatGPT-direct_assessment_score_response,ChatGPT-direct_assessment_score_label,ChatGPT-direct_assessment_score_response_afacts,ChatGPT-direct_assessment_score_label_afacts,Alpaca-direct_assesment_response,Alpaca-direct_assesment_label
0,0,40,40,13681241,MV-BART,[],[],Callum is still busy and sorry.,0,[],...,yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
1,1,41,41,13681241,CODS,[],[],Callum is busy.,0,[],...,Yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
2,2,42,42,13681241,UniLM,[],[],Callum is still busy .,0,[],...,Yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
3,3,43,43,13681241,BART,[],[],Callum is busy.,0,[],...,Yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
4,4,44,44,13681241,gpt4-32k-0613,[],[],"Jair checks if Callum is still busy, and Callu...",0,[],...,Yes,0,Yes,0,100.0,100,90.0,90,Decide if the following summary is consistent ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,1945,2023,2023,test_339,CODS,"[""and #Person1#'s mom "", '#Person1# and #Perso...",[],#Person1# and #Person2# are talking about what...,1,"['Intrinsic_Error', 'Intrinsic_Error']",...,Yes,0,Yes,0,90.0,90,60.0,60,Decide if the following summary is consistent ...,1
1946,1946,2024,2024,test_339,UniLM,['person1 and person2 '],[],person1 and person2 are asking each other abou...,1,['Intrinsic_Error'],...,Yes,0,Yes,0,80.0,80,70.0,70,Decide if the following summary is consistent ...,1
1947,1947,2025,2025,test_339,BART,['##Person1# and #Person2# '],[],##Person1# and #Person2# are talking about wha...,1,['Intrinsic_Error'],...,Yes,0,Yes,0,100.0,100,80.0,80,Decide if the following summary is consistent ...,1
1948,1948,2026,2026,test_339,gpt4-32k-0613,[],[],Person1 and Person2 are asking each other hypo...,0,[],...,yes,0,yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1


In [13]:
df_sample_nonfactual

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,docid,model,nonfactual_spans,evidence,summary,factual_error,error_type,...,ChatGPT-direct_assesment_response,ChatGPT-direct_assesment_label,ChatGPT-direct_assesment_response_afacts,ChatGPT-direct_assesment_label_afacts,ChatGPT-direct_assessment_score_response,ChatGPT-direct_assessment_score_label,ChatGPT-direct_assessment_score_response_afacts,ChatGPT-direct_assessment_score_label_afacts,Alpaca-direct_assesment_response,Alpaca-direct_assesment_label
1348,1348,1950,1950,13731225,gpt4-32k-0613,['asks to wait for him to have dinner'],['Julia : I was wondering if we should wait f...,Bert tells Julia he will be home around 8 and ...,1,['Intrinsic_Error'],...,Yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
1777,1777,1551,1551,test_43,CODS,['and listen '],[],#Person2# is bored during the speech. #Person1...,1,['Intrinsic_Error'],...,no,1,No,1,95.0,95,90.0,90,Decide if the following summary is consistent ...,1


In [12]:
#### nonfactual examples ###
df_scored_factual = df_scored[df_scored['factual_error'] == 0]
df_sample_factual = df_scored_factual.sample(2)
drop_idx = list(df_sample_factual.index)
df_scored.drop(index=drop_idx)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,docid,model,nonfactual_spans,evidence,summary,factual_error,error_type,...,ChatGPT-direct_assesment_response,ChatGPT-direct_assesment_label,ChatGPT-direct_assesment_response_afacts,ChatGPT-direct_assesment_label_afacts,ChatGPT-direct_assessment_score_response,ChatGPT-direct_assessment_score_label,ChatGPT-direct_assessment_score_response_afacts,ChatGPT-direct_assessment_score_label_afacts,Alpaca-direct_assesment_response,Alpaca-direct_assesment_label
0,0,40,40,13681241,MV-BART,[],[],Callum is still busy and sorry.,0,[],...,yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
1,1,41,41,13681241,CODS,[],[],Callum is busy.,0,[],...,Yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
2,2,42,42,13681241,UniLM,[],[],Callum is still busy .,0,[],...,Yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
3,3,43,43,13681241,BART,[],[],Callum is busy.,0,[],...,Yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
4,4,44,44,13681241,gpt4-32k-0613,[],[],"Jair checks if Callum is still busy, and Callu...",0,[],...,Yes,0,Yes,0,100.0,100,90.0,90,Decide if the following summary is consistent ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,1945,2023,2023,test_339,CODS,"[""and #Person1#'s mom "", '#Person1# and #Perso...",[],#Person1# and #Person2# are talking about what...,1,"['Intrinsic_Error', 'Intrinsic_Error']",...,Yes,0,Yes,0,90.0,90,60.0,60,Decide if the following summary is consistent ...,1
1946,1946,2024,2024,test_339,UniLM,['person1 and person2 '],[],person1 and person2 are asking each other abou...,1,['Intrinsic_Error'],...,Yes,0,Yes,0,80.0,80,70.0,70,Decide if the following summary is consistent ...,1
1947,1947,2025,2025,test_339,BART,['##Person1# and #Person2# '],[],##Person1# and #Person2# are talking about wha...,1,['Intrinsic_Error'],...,Yes,0,Yes,0,100.0,100,80.0,80,Decide if the following summary is consistent ...,1
1948,1948,2026,2026,test_339,gpt4-32k-0613,[],[],Person1 and Person2 are asking each other hypo...,0,[],...,yes,0,yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1


In [16]:
pd.concat([df_sample_factual, df_sample_factual])

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,docid,model,nonfactual_spans,evidence,summary,factual_error,error_type,...,ChatGPT-direct_assesment_response,ChatGPT-direct_assesment_label,ChatGPT-direct_assesment_response_afacts,ChatGPT-direct_assesment_label_afacts,ChatGPT-direct_assessment_score_response,ChatGPT-direct_assessment_score_label,ChatGPT-direct_assessment_score_response_afacts,ChatGPT-direct_assessment_score_label_afacts,Alpaca-direct_assesment_response,Alpaca-direct_assesment_label
1210,1210,1582,1582,13819472,gpt4-32k-0613,[],[],"Jenny, Jeremy, and Joan plan to attend a conce...",0,[],...,yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
849,849,843,843,13728157,mv-bart_large,[],[],Callan has a problem with his Samsung S8. It o...,0,[],...,Yes,0,Yes,0,100.0,100,90.0,90,Decide if the following summary is consistent ...,1
1210,1210,1582,1582,13819472,gpt4-32k-0613,[],[],"Jenny, Jeremy, and Joan plan to attend a conce...",0,[],...,yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
849,849,843,843,13728157,mv-bart_large,[],[],Callan has a problem with his Samsung S8. It o...,0,[],...,Yes,0,Yes,0,100.0,100,90.0,90,Decide if the following summary is consistent ...,1
